In [ ]:
#import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [ ]:
deeplabv3plus_dir="./src"
sys.path.append(deeplabv3plus_dir)

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="3", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [ ]:
#from model import deeplab_v3plus
from image_utils import make_x_from_image_paths,make_y_from_image_paths,convert_y_to_image_array, make_palette
from data_gen import DataGenerator

In [ ]:
n_categories = 21
image_size = (512,512)

In [ ]:
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

img_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))]
seg_img_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))]

valid_x_paths = [os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))]
valid_y_paths = [os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))]
#valid_x_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))][0:10]
#valid_y_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))][0:10]

In [ ]:
batch_size = 8
preprocess = keras.applications.xception.preprocess_input

In [ ]:
train_data_gen = DataGenerator(n_categories, image_size, batch_size, img_paths    , seg_img_paths, preprocess, augmentation=True , shuffle=True)

In [ ]:
x,y = train_data_gen.__getitem__(0)

In [ ]:
x = (x+1)/2
y =convert_y_to_image_array(y, image_size, n_categories, palette)

In [ ]:
i=4
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(x[i,:,:,:])
plt.subplot(1,2,2)
plt.imshow(y[i])

# Augmentor test

In [ ]:
from albumentations import *

In [ ]:
x = make_x_from_image_paths(img_paths[0:10], image_size)
y = make_y_from_image_paths(seg_img_paths[0:10], image_size, n_categories)

In [ ]:
i=1
#aug=RandomCrop(p=1, height=300, width=300)
#aug=ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=1)
aug=Compose([HueSaturationValue(p=1)],p=1)
img=aug(image=x[i,:,:,:])["image"]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(x[i,:,:,:])
plt.subplot(1,2,2)
plt.imshow(img)